In [180]:
import os

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch import optim

In [181]:
# Hyperparameters
EPOCHS = 200
BATCH = 100
LEARNING_RATE = 0.001

In [182]:
path = os.path.join('data', 'train', '2015-data.csv')
df = pd.read_csv(path, index_col=['GAME_ID', 'TEAM_ID'])

x = torch.tensor(
    df[df.columns.difference(['TEAM_PTS'])].values,
    dtype=torch.float32
)
y = torch.tensor(
    df['TEAM_PTS'].values, 
    dtype=torch.float32
).unsqueeze(dim=1)

In [183]:
class NeuralNet(nn.Module):
    
    def __init__(self):
        super(NeuralNet, self).__init__()
        # an affine operation: y = Wx + b
        self.lin1 = nn.Linear(in_features=144, out_features=128)
        self.lin2 = nn.Linear(128, 64)
        self.lin3 = nn.Linear(64, 16)
        self.lin4 = nn.Linear(16, 4)
        self.lin5 = nn.Linear(4, 1)
        
    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        x = F.relu(self.lin4(x))
        x = self.lin5(x)
        return x

In [184]:
def get_model():
    model = NeuralNet()
    return model, optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [185]:
# Follows https://pytorch.org/tutorials/beginner/nn_tutorial.html#refactor-using-optim
model, opt = get_model()

train_ds = TensorDataset(x, y)
train_dl = DataLoader(train_ds, batch_size=BATCH)

loss_func = F.l1_loss

for _ in range(EPOCHS):
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        loss.backward()
        opt.step()
        opt.zero_grad()
        
print(loss_func(model(xb), yb))

tensor(15.3174, grad_fn=<L1LossBackward>)
